In [ ]:
import json
import numpy as np

RESULTS_PATH = "results/"
METRICS_FILE = "metrics.json"

METRICS_PATH = RESULTS_PATH + METRICS_FILE
PLOTS_OUT_PATH = "plots/"

max = np.max(1,2)
print(max)
with open(METRICS_PATH, "r") as f:
    file = open(METRICS_PATH)
    metrics = list(map(lambda x: json.loads(x), file.readlines()))

In [ ]:
import sys
print(sys.executable)

In [ ]:
avg_reply = np.mean(list(map(lambda x: int(x["operation_reply"]), metrics)))
print("Average reply time: " + str(avg_reply) + " ms")